In [4]:
import sys
sys.path.append('/Users/lindsaybassman/Documents/LBL/ArQTiC')
from arqtic.simulation_generator import Simulation_Generator
import matplotlib.pyplot as plt
import numpy as np
import qiskit as qk
from qiskit import Aer, IBMQ, execute
import random

In [9]:
#create simulation object
sim_obj = Simulation_Generator("QITE_input.txt")

#Pauli matrices
sx = np.array([[0, 1], [1, 0]])
sy = np.array([[0, -1j], [1j, 0]])
sz = np.array([[1, 0], [0, -1]])
#setup inital hamiltonian

#number of samples
T = 10
#number of qubits
N = sim_obj.num_spins
#size of matrix
msize = 2**N
#Hamiltonian parameters
Jz = sim_obj.Jz #ising interaction strength !!! Not used for 1-qubit
mu_x = sim_obj.h_ext #transverse magnetic field strength
#initial Hamiltonian
hinit = np.zeros((msize, msize), dtype=np.complex128)
hinit+=-Jz*np.kron(sz,sz)
hinit+=-mu_x*np.kron(sx,np.eye(2))
hinit+=-mu_x*np.kron(np.eye(2),sx)
#hinit += (-Jz)*np.kron(np.eye(2), np.kron(sz, sz))
#hinit += (-Jz)*np.kron(sz, np.kron(sz, np.eye(2)))
#hinit += (-mu_x)*np.kron(np.eye(2), np.kron(np.eye(2), sx))
#hinit += (-mu_x)*np.kron(np.eye(2), np.kron(sx,np.eye(2)))
#hinit += (-mu_x)*np.kron(sx, np.kron(np.eye(2), np.eye(2)))


#Quantum METTS
#set up simulators
qasm_sim = Aer.get_backend('qasm_simulator')
state_vector_sim = Aer.get_backend('statevector_simulator')
shots = 2048

#get random first product state for METTS
measured_metts_state = []
for _ in range(N):
    measured_metts_state.append(random.choice([0,1]))
    
#for METTS we want to evolve by beta/2 for system at temperature beta
sim_obj.beta = sim_obj.beta/2
#loop over trajectories
eng_array = []
q_regs = qk.QuantumRegister(N, 'q')
c_regs = qk.ClassicalRegister(N, 'c')
for i in range(T):
    #print(i)
    #set initial product state
    print(measured_metts_state)
    sim_obj.initial_spins = measured_metts_state
    #note QITE algorithm should evolve state by beta/2 for temperature beta
    sim_obj.generate_programs()
    qite_circ = sim_obj.ibm_circuits_list[0]
    energies = sim_obj.qite_energies
    print(energies[-1])
    #measure energy of initial Hamiltonian
    eng_circ = qite_circ.copy()
    #eng_circ = qk.QuantumCircuit(q_regs, c_regs)
    #for q in range(N):
    #    if (measured_metts_state[q] == 1):
    #        eng_circ.x(q)
    #eng_circ.compose(qite_circ, inplace=True)
    result = qk.execute(eng_circ, state_vector_sim).result()
    statevector = result.get_statevector(eng_circ)
    eng = np.real(np.dot(np.conj(np.transpose(statevector)), np.dot(hinit, statevector)))
    print(eng)
    eng_array.append(eng)
    #get measured_metts_state for next trajectory
    METTS_circ = eng_circ.copy()
    if (i%2 == 0):
        METTS_circ.h(q_regs)  
    #add measurement operators
    METTS_circ.measure(q_regs, c_regs)
    result = qk.execute(METTS_circ, qasm_sim, shots=1).result().get_counts()
    for spin_str, count in result.items():
        for q in range(N):
            measured_metts_state[q] = int(spin_str[q])

[1, 1]
Creating IBM quantum circuit objects...
IBM quantum circuit objects created
-0.709590018048349
-0.45372001691401204
[1, 0]
Creating IBM quantum circuit objects...
IBM quantum circuit objects created
-0.777951021541706
-0.45372001691401204
[0, 0]
Creating IBM quantum circuit objects...
IBM quantum circuit objects created
-0.777951021541706
-0.45372001691401204
[1, 0]
Creating IBM quantum circuit objects...
IBM quantum circuit objects created
-0.777951021541706
-0.45372001691401204
[0, 0]
Creating IBM quantum circuit objects...
IBM quantum circuit objects created
-0.777951021541706
-0.45372001691401204
[0, 1]
Creating IBM quantum circuit objects...
IBM quantum circuit objects created
-0.709590018048349
-0.45372001691401204
[0, 0]
Creating IBM quantum circuit objects...
IBM quantum circuit objects created
-0.777951021541706
-0.45372001691401204
[0, 1]
Creating IBM quantum circuit objects...
IBM quantum circuit objects created
-0.709590018048349
-0.45372001691401204
[0, 0]
Creating 

In [10]:
for t in [10,100,200,500,750,1000]:
    print(np.mean(eng_array[0:t]))

-0.453720016914012
-0.453720016914012
-0.453720016914012
-0.453720016914012
-0.453720016914012
-0.453720016914012


In [4]:
print(eng_circ.qasm())

OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg c[2];
x q[0];
gate unitary140248279543712 p0,p1 {
	u3(0.80460098,-0.27541132,4.9058868) p0;
	u3(2.3369917,-3.417004,1.3772985) p1;
	cx p0,p1;
	u3(2.7545971,0,3*pi/2) p0;
	u3(pi/2,-pi/2,1.1838007) p1;
	cx p0,p1;
	u3(2.3369917,1.7642941,-1.295385) p0;
	u3(1.3735707,0,-pi/4) p1;
}
unitary140248279543712 q[0],q[1];
gate unitary140248279708048 p0,p1 {
	u3(2.1028249,-2.3734506,-1.1151239) p0;
	u3(2.2392771,2.2736714,-2.2023954) p1;
	cx p0,p1;
	u3(2.9236776,0,3*pi/2) p0;
	u3(pi/2,-pi/2,1.3528812) p1;
	cx p0,p1;
	u3(1.1835708,0.98945199,2.6199949) p0;
	u3(2.5028231,-pi,-0.88489476) p1;
}
unitary140248279708048 q[0],q[1];
gate unitary140248279709104 p0,p1 {
	u3(1.3051965,-1.8462949,-2.3191614) p0;
	u3(1.8363962,1.2952978,-3.9640239) p1;
	cx p0,p1;
	u3(3.0344908,0,3*pi/2) p0;
	u3(pi/2,-pi/2,1.4636945) p1;
	cx p0,p1;
	u3(1.7317975,2.343005,2.9784633) p0;
	u3(2.9128967,-pi,-3*pi/4) p1;
}
unitary140248279709104 q[0],q[1];
gate unitary1402482797093